<a href="https://colab.research.google.com/github/almiab1/MUIIADeepLearning/blob/main/PracticaFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica Final | Deep Learning

En el presente trabajo se proponen dos modelos basados en redes convolucionales, los cuales se trata de clasificar correctamente si un individuo está parpadeando o no. En la actualidad se pueden encontrar modelos de gran calidad, los cuales permiten construir modelos de gran precisión basándose sobre la base de dichos modelos pre-entrenados. Por ello, los modelos propuestos en este trabajo se definen haciendo uso de modelos previamente entrenados.

En primer lugar, se propone una red basada en el modelo `InceptionResNetV2`. Un modelo pesado de gran profundidad que proporciona buenos resultados en problemas de clasificación. Se propone una estrategia donde el modelo `InceptionResNetV2` toma la función de extractor de características, obteniendo así las características profundas que posteriormente podrán ser utilizadas en la clasificación de las imágenes. En segundo punto, se propone una red basada en el modelo `DenseNet`. Un modelo de menor tamaño y que permite obtener resultados similares a los de `InceptionResNetV2`. En este caso se trata de re-entrenar el clasificador y aplicando `fine tuning` con el fin de ajustar los parámetros del modelo.

Dado que se parte de un conjunto de datos desbalanceado, es necesario aplicar algunos métodos de balanceo con el fin de obtener un modelo más robusto. Previamente, al entrenamiento de la red, se propone el uso de técnicas como la estratificación de datos o `data augmentation`. A su vez, para mejorar el proceso de entrenamiento se hace empleo de `mini-batches` y de optimizadores como el optimizador Adam.

In [1]:
import pandas as pd
import numpy as np

from tensorflow_addons.metrics import F1Score 

from sklearn.model_selection import train_test_split

# Keras imports
from tensorflow.keras import optimizers, Input

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Concatenate, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras.applications import InceptionResNetV2, DenseNet169 

from tensorflow.keras.preprocessing.image import ImageDataGenerator


/Users/aijutic/miniconda3/envs/cnnDL/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
_isColab = False

if _isColab:
    from google.colab import drive

    # Montamos el Google Drive en el directorio del proyecto y descomprimios el fichero con los datos
    drive.mount('/content/drive')
    !unzip -n '/content/drive/MyDrive/UIMP/Asignaturas/5-DeepLearning/Practicas/Final/Source/RT-BENE.zip' >> /dev/null

## Pasos previos

### Preparar el conjunto de datos

*   **Cargar el dataset en el colab**

In [3]:
# Especificamos los paths al directorio que contiene las imagenes y al fichero con las etiquetas
data_path = 'RT-BENE/'

if _isColab == False:
    data_path = "./../RT-BENE/"

imgs_path = data_path + "images/"
labels_path = data_path + "blinks.csv"


# Leemos el fichero CSV con las etiquetas
labels = pd.read_csv(labels_path, dtype = {"class": "category"})

# Supongamos que tienes un DataFrame llamado 'labels' con los nombres de las imágenes y las etiquetas 'blink'
# Convertir la columna 'blink' a tipo string
labels['blink'] = labels['blink'].astype(str)

# Mostramos los primero elementos del dataset
labels.head()

,blink_id,left_eye,right_eye,video,blink
0,0,0_left_000001_rgb.png,0_right_000001_rgb.png,0,0
1,1,0_left_000002_rgb.png,0_right_000002_rgb.png,0,0
2,2,0_left_000003_rgb.png,0_right_000003_rgb.png,0,0
3,3,0_left_000004_rgb.png,0_right_000004_rgb.png,0,0
4,4,0_left_000005_rgb.png,0_right_000005_rgb.png,0,0


- **Creamos las tres particiones de datos: entrenamiento, validación y test**


In [4]:
# Semilla para replicar los experimentos
seed = 2023
# Creamos las tres particiones de datos: entrenamiento, validación y test
train_data, test_data = train_test_split(labels, test_size=0.2, random_state=seed)
dev_data, test_data = train_test_split(test_data, test_size=0.5, random_state=seed)

En Keras no existen generadores por defecto que devuelvan dos imágenes. Necesitamos crear nuestro propio generador que devuelva a la vez las imágenes de ambos ojos y la etiqueta del frame. Para ello podemos crear dos generadores (uno para cada ojo) usando el método `flow_from_databrame` y combinarlos para crear el generador deseado.

- En primer lugar se hace uso del método `ImageDataGenerator` para preprocesar imágenes y generar lotes de datos de entrenamiento y validación.
- De seguido se definen los generadores de datos

In [5]:
# Aplicar tecnicas de preprocesado
datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, # rotación aleatoria en un rango de 20 grados
    width_shift_range=0.1, # desplazamiento horizontal aleatorio en un rango del 10% de la anchura de la imagen
    height_shift_range=0.1, # desplazamiento vertical aleatorio en un rango del 10% de la altura de la imagen
    zoom_range=0.1, # zoom aleatorio en un rango del 10%
    horizontal_flip=True, # reflejo horizontal aleatorio
    vertical_flip=False) # reflejo horizontal aleatorio

# Columnas dataset
left_eye_col = 'left_eye'
right_eye_col = 'right_eye'
y_col = 'blink'

# Parámetros
batch_size = 128
img_width = 122 # minimo 75x75
img_height = 75

# Generador custom que devuelve las dos imagenes de ojos y el label del parpadeo
def generator(dataframe):
  left_eye_generator = datagen.flow_from_dataframe(dataframe=dataframe, 
                                                    directory = imgs_path, 
                                                    target_size =(img_width, img_height), 
                                                    x_col=left_eye_col, 
                                                    y_col=y_col, 
                                                    class_mode="binary", 
                                                    seed=seed, 
                                                    batch_size=batch_size)
  
  right_eye_generator = datagen.flow_from_dataframe(dataframe=dataframe,
                                                    directory = imgs_path,
                                                    target_size =(img_width, img_height),
                                                    x_col=right_eye_col,
                                                    y_col=y_col,
                                                    class_mode="binary",
                                                    seed=seed,
                                                    batch_size=batch_size)
  
  while True:
    left_eye = left_eye_generator.next()
    left_eye_image = left_eye[0]
    label = left_eye[1]
    right_eye = right_eye_generator.next()
    right_eye_image = right_eye[0]
    yield [left_eye_image, right_eye_image], label

# Llamada a la función generator
train_generator = generator(train_data)
dev_generator = generator(dev_data)
test_generator = generator(test_data)

- **Definición de parámetros comunes**

In [6]:
"""" Configuraciones previas """

# Definimos la métrica F1
f1_score = F1Score(name="f1_score", num_classes=1, threshold=0.5, average='macro')

# Declaramos dos capas de Input
input_shape = (img_width,img_height,3)

input_image1 = Input(shape=input_shape)
input_image2 = Input(shape=input_shape)

# Definimos el ratio de aprendizaje
lr = 1e-4

Metal device set to: Apple M1 Ultra

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB



## Trasnfer learing con IceptionResNetV2 como extractor de características

Antes de entrenar el modelo, es importante decidir qué capas se actualizarán durante el entrenamiento. Si deseas aplicar fine-tuning en algunas de las capas del modelo base, primero puedes congelar todas las capas y luego descongelar las últimas capas.

In [7]:
# Cargar InceptionResNetV2 pre-entrenado en ImageNet, sin la capa final
base_model_incept = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')

# Congelar todas las capas del modelo base para aplicar transfer learning
for layer in base_model_incept.layers:
    layer.trainable = False

# Extraer las características utilizando InceptionResNetV2
features_image1 = base_model_incept(input_image1)
features_image2 = base_model_incept(input_image2)

# Combinar las características de ambas imágenes
merged_features = Concatenate()([features_image1, features_image2])

# Añadir capas adicionales para la clasificación
x = Dense(1024, activation='relu')(merged_features)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

# Crear el modelo
model_incept = Model(inputs=[input_image1, input_image2], outputs=output)

# Compilar el modelo
model_incept.compile(optimizer=optimizers.Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy', f1_score])

# Imprimir un resumen del modelo
model_incept.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 122, 75, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 input_2 (InputLayer)           [(None, 122, 75, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 inception_resnet_v2 (Functiona  (None, 1536)        54336736    ['input_1[0][0]',                
 l)                                                               'input_2[0][0]']            

In [8]:
# Configuración del método 'Early stopping'
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenamos el modelo con los datos preparados en el punto 2
model_incept.fit(train_generator,
          epochs=10,  # numero de epochs
          verbose=2,  # muestra informacion del error al finalizar cada epoch
          steps_per_epoch=len(train_data)/batch_size,
          validation_data=train_generator,
          validation_steps=len(dev_data)/batch_size,
          callbacks=[early_stopping]) # Añadir el callback de 'early stoppoing'

Found 85880 validated image filenames belonging to 2 classes.
Found 85880 validated image filenames belonging to 2 classes.
Epoch 1/10


2023-04-05 17:14:32.448808: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


670/670 - 110s - loss: 0.0572 - accuracy: 0.9807 - f1_score: 0.7395 - val_loss: 0.0282 - val_accuracy: 0.9905 - val_f1_score: 0.8789 - 110s/epoch - 164ms/step
Epoch 2/10
670/670 - 95s - loss: 0.0310 - accuracy: 0.9897 - f1_score: 0.8715 - val_loss: 0.0216 - val_accuracy: 0.9924 - val_f1_score: 0.9021 - 95s/epoch - 141ms/step
Epoch 3/10
670/670 - 96s - loss: 0.0241 - accuracy: 0.9917 - f1_score: 0.9004 - val_loss: 0.0207 - val_accuracy: 0.9927 - val_f1_score: 0.9095 - 96s/epoch - 143ms/step
Epoch 4/10
670/670 - 95s - loss: 0.0213 - accuracy: 0.9929 - f1_score: 0.9114 - val_loss: 0.0172 - val_accuracy: 0.9946 - val_f1_score: 0.9316 - 95s/epoch - 142ms/step
Epoch 5/10
670/670 - 95s - loss: 0.0184 - accuracy: 0.9937 - f1_score: 0.9250 - val_loss: 0.0185 - val_accuracy: 0.9943 - val_f1_score: 0.9271 - 95s/epoch - 141ms/step
Epoch 6/10
670/670 - 94s - loss: 0.0167 - accuracy: 0.9942 - f1_score: 0.9307 - val_loss: 0.0164 - val_accuracy: 0.9947 - val_f1_score: 0.9319 - 94s/epoch - 140ms/step
E

Después del entrenamiento, evalúa el rendimiento del modelo en el conjunto de prueba:

In [9]:
# Por ultimo, podemos evaluar el modelo en el conjunto de test
test_loss, test_acc, test_f1_score = model_incept.evaluate(test_generator, steps=len(test_data) / batch_size, verbose=1)
print(f"Test_loss: {test_loss:.4f}, Test_acc: {test_acc:.4f}, Test_F1_Score: {test_f1_score:.4f}")


Found 85880 validated image filenames belonging to 2 classes.
Found 85880 validated image filenames belonging to 2 classes.
83/83 [==============================] - 11s 129ms/step - loss: 0.0071 - accuracy: 0.9978 - f1_score: 0.9729
Test_loss: 0.0071, Test_acc: 0.9978, Test_F1_Score: 0.9729


Además de analizar el error obtenido, podemos utilizarlo para hacer predicciones. Para ello utilizaremos el método `predict`, al que le suministremos los datos de entrada.

In [ ]:
# Obtenemos las predicciones para todos los ejemplos del conjunto de test
predictions = model_incept.predict(test_generator, verbose=1)

# Imprimimos la predicción obtenida para los dos primeros ejemplos
# Los valores obtenidos representan las probabilidades para cada una de las 5 clases
for i in range(0,2):
  print("\n Ejemplo", i)
  print("\t Probabilidades para las 5 clases:", predictions[i])
  print("\t Clase predicha: %i, Probabilidad: %.4f" % (np.argmax([predictions[i]]), np.max(predictions[i])))

## Fine Tuning con DenseNet

En este caso se hace uso del modelo **DenseNet121**. Este tiene un tamaño de 80 MB, ha sido entrenado con 20.1 millones de parámetros y tiene una profuncidad de 402 capas. Respecto a la precisión del mismo, presenta un 93.6% de precisión para el dataset de validación de ImageNet.

In [11]:
# Cargar el modelo DenseNet121 pre-entrenado en ImageNet sin incluir la capa de salida
base_model_densenet = DenseNet169(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Congelar todas las capas del modelo base
for layer in base_model_densenet.layers:
    layer.trainable = False

# Descongelar las últimas capas para aplicar fine-tuning
# (en este ejemplo, descongelamos las últimas 50 capas)
for layer in base_model_densenet.layers[-50:]:
    layer.trainable = True

# Añadir una capa de GlobalAveragePooling2D
x = base_model_densenet.output
x = GlobalAveragePooling2D()(x)

# Añadir una capa Dense de salida con una función de activación sigmoide (clasificación binaria)
output = Dense(1, activation='sigmoid')(x)

# Crear el nuevo modelo
model_dn = Model(inputs=[input_image1, input_image2], outputs=output)

# Compilar el modelo con la función de pérdida binary_crossentropy y la métrica de precisión
model_dn.compile(optimizer=optimizers.Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])

51877672/51877672 [==============================] - 3s 0us/step


- Entrenar el modelo

In [12]:
# Configuración del método 'Early stopping'
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenamos el modelo con los datos preparados en el punto 2
model_dn.fit(train_generator,
          epochs=10,  # numero de epochs
          verbose=2,  # muestra informacion del error al finalizar cada epoch
          steps_per_epoch=len(train_data)/batch_size,
          validation_data=train_generator,
          validation_steps=len(dev_data)/batch_size,
          callbacks=[early_stopping]) # Añadir el callback de 'early stoppoing'

Epoch 1/10
670/670 - 175s - loss: 0.0099 - accuracy: 0.9966 - f1_score: 0.9598 - val_loss: 0.0055 - val_accuracy: 0.9985 - val_f1_score: 0.9831 - 175s/epoch - 261ms/step
Epoch 2/10
670/670 - 140s - loss: 0.0105 - accuracy: 0.9964 - f1_score: 0.9567 - val_loss: 0.0136 - val_accuracy: 0.9948 - val_f1_score: 0.9372 - 140s/epoch - 209ms/step
Epoch 3/10
670/670 - 140s - loss: 0.0099 - accuracy: 0.9966 - f1_score: 0.9604 - val_loss: 0.0044 - val_accuracy: 0.9986 - val_f1_score: 0.9829 - 140s/epoch - 208ms/step
Epoch 4/10
670/670 - 132s - loss: 0.0084 - accuracy: 0.9970 - f1_score: 0.9637 - val_loss: 0.0067 - val_accuracy: 0.9977 - val_f1_score: 0.9730 - 132s/epoch - 196ms/step
Epoch 5/10
670/670 - 135s - loss: 0.0082 - accuracy: 0.9972 - f1_score: 0.9660 - val_loss: 0.0048 - val_accuracy: 0.9981 - val_f1_score: 0.9764 - 135s/epoch - 201ms/step
Epoch 6/10
670/670 - 139s - loss: 0.0083 - accuracy: 0.9972 - f1_score: 0.9661 - val_loss: 0.0054 - val_accuracy: 0.9980 - val_f1_score: 0.9761 - 139s

Después del entrenamiento, evalúa el rendimiento del modelo en el conjunto de prueba:

In [13]:
# Por ultimo, podemos evaluar el modelo en el conjunto de test
test_loss, test_acc, test_f1_score = model_dn.evaluate(test_generator, steps=len(test_data) / batch_size, verbose=1)
print(f"Test_loss: {test_loss:.4f}, Test_acc: {test_acc:.4f}, Test_F1_Score: {test_f1_score:.4f}")

ValueError: in user code:

    File "/Users/aijutic/miniconda3/envs/cnnDL/lib/python3.10/site-packages/keras/engine/training.py", line 1852, in test_function  *
        return step_function(self, iterator)
    File "/Users/aijutic/miniconda3/envs/cnnDL/lib/python3.10/site-packages/keras/engine/training.py", line 1836, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/aijutic/miniconda3/envs/cnnDL/lib/python3.10/site-packages/keras/engine/training.py", line 1824, in run_step  **
        outputs = model.test_step(data)
    File "/Users/aijutic/miniconda3/envs/cnnDL/lib/python3.10/site-packages/keras/engine/training.py", line 1788, in test_step
        y_pred = self(x, training=False)
    File "/Users/aijutic/miniconda3/envs/cnnDL/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/aijutic/miniconda3/envs/cnnDL/lib/python3.1

Además de analizar el error obtenido, podemos utilizarlo para hacer predicciones. Para ello utilizaremos el método `predict`, al que le suministremos los datos de entrada.

In [ ]:
# Obtenemos las predicciones para todos los ejemplos del conjunto de test
predictions = model_incept.predict(test_generator, verbose=1)

# Imprimimos la predicción obtenida para los dos primeros ejemplos
# Los valores obtenidos representan las probabilidades para cada una de las 5 clases
for i in range(0,2):
  print("\n Ejemplo", i)
  print("\t Probabilidades para las 5 clases:", predictions[i])
  print("\t Clase predicha: %i, Probabilidad: %.4f" % (np.argmax([predictions[i]]), np.max(predictions[i])))